<a href="https://www.kaggle.com/code/ngocthuy/da-jobs-in-canada-random-forest-plotly?scriptVersionId=194285575" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from urllib.request import urlopen
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,accuracy_score
from sklearn.pipeline import Pipeline
from category_encoders import BinaryEncoder



#  **1. EXPLORATORY DATA ANALYSIS**

In [ ]:
df = pd.read_csv('/kaggle/input/data-analyst-job-roles-in-canada/Cleaned_Dataset.csv')
df.head(5)

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.describe()

In [ ]:
fig = px.box(df, y =['Min_Salary', 'Max_Salary', 'Avg_Salary'])
fig.update_layout(
    title = 'Salary Distribution (Min, Max, Avg)',
    yaxis_title= 'Salary',
    xaxis_title='Salary Type',
    boxmode='group')

fig.show()

In [ ]:
df.nunique()

# **2. DATA VISUALIZATION**

# 2.1. Distribution and Avg Salary by Job Title, Job Position, Seniority

In [ ]:
Count_JobTitle = df['Job Title'].value_counts().reset_index()

fig = px.bar(Count_JobTitle, x = 'Job Title', y = 'count', title = 'Distribution of Job Title',
            hover_data=['Job Title', 'count'], color = 'count',
            labels={'count':'Count of Job Title'}, height = 600)
fig.show()

In [ ]:
Count_Position = df['Position'].value_counts().reset_index()

fig = px.bar(Count_Position, x = 'Position', y = 'count', title = 'Distribution of Position',
            hover_data=['Position', 'count'], color = 'count',
            labels={'count':'Count of Position'}, height = 600)
fig.show()

In [ ]:
# Salary distribution by job roles
fig = px.box(df, x = 'Job Title', y = 'Avg_Salary')
fig.show()

In [ ]:
# Avg Salary based on Seniority

fig1 = go.Box(x=df[df['Seniority']!='ANY']['Seniority'], y=df[df['Seniority']!='ANY']['Avg_Salary'])
df_sub = df[df['Seniority']!='ANY'].groupby('Seniority').agg(Avg_Salary=('Avg_Salary', 'mean')).reset_index()
fig2 = go.Bar(x = df_sub['Seniority'], y=df_sub['Avg_Salary'], marker=dict(color='skyblue'), width=0.5)

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('Distribution of Avg Salary by Seniority', 'Avg Salary by Seniority'), 
                    horizontal_spacing=0.15)

fig.add_trace(fig1, row=1, col=1)
fig.add_trace(fig2, row=1, col=2)
fig.update_layout(height=600, width=1200, title_text='Distribution of Avg Salary by Seniority')
fig.show()


# 2.2. Avg Salary by Employers

In [ ]:
# Count of total Data Analyst jobs by Employers
Count_Employers = df['Employer'].value_counts().reset_index().sort_values(by='count', ascending=False)
Count_Employers.head(10)

In [ ]:
# Which companies pay highest salary?
df_senior = df[df['Seniority']=='Senior'].groupby('Employer').agg(Avg_Salary=('Avg_Salary', 'mean'))\
                                .reset_index().sort_values(by='Avg_Salary', ascending=False)\
                                .head(20).sort_values(by='Avg_Salary', ascending=True)
df_junior= df[df['Seniority']=='Junior'].groupby('Employer').agg(Avg_Salary=('Avg_Salary', 'mean'))\
                                .reset_index().sort_values(by='Avg_Salary', ascending=False)\
                                .head(20).sort_values(by='Avg_Salary', ascending=True)

fig_senior = go.Bar(y=df_senior['Employer'], x=df_senior['Avg_Salary'], name='Senior', orientation='h')
fig_junior = go.Bar(y=df_junior['Employer'], x=df_senior['Avg_Salary'], name='Junior',  orientation='h')

fig = make_subplots(rows=1, cols=2, subplot_titles=('Senior Positions', 'Junior Positions'), horizontal_spacing=0.25)
fig.add_trace(fig_senior, row=1, col=1)
fig.add_trace(fig_junior, row=1, col=2)
fig.update_layout(height=600, width=1200, title_text='Top 20 Employers pay highest salary for Senior and Junior Positions')
fig.show()



# 2.3. Avg Salary based on Location ( Provinces, Cities)

In [ ]:
# Count of total Data Analyst jobs by Cities
Count_City = df['City'].value_counts().reset_index().sort_values(by='count', ascending=False)
Count_City

In [ ]:
Count_province = df[df['Province'] != 'Undef']['Province'].value_counts().reset_index().sort_values(by='count', ascending=False)
Count_province.columns = ['Province', 'Count']
Count_province

province_map = {
    'ON': 'Ontario',
    'BC': 'British Columbia',
    'AB': 'Alberta',
    'QC': 'Quebec',
    'MB': 'Manitoba',
    'NS': 'Nova Scotia',
    'NB': 'New Brunswick',
    'SK': 'Saskatchewan',
    'NL': 'Newfoundland and Labrador',
    'YT': 'Yukon',
    'PE': 'Prince Edward Island',
    'NT': 'Northwest Territories',
    'NFL': 'Newfoundland and Labrador'  # Assuming 'NFL' refers to Newfoundland and Labrador
}
df['Province_FN'] = df["Province"].map(province_map)
Count_province['Province']=Count_province['Province'].map(province_map)
Count_province




In [ ]:
#Job Count by Province in Canada

with urlopen('https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/canada.geojson') as response:
    provinces = json.load(response)
#     print(provinces['features'])
fig = px.choropleth(Count_province, geojson=provinces,locations='Province',
                   featureidkey='properties.name',color='Count',
                    color_continuous_scale="Reds",
                    range_color=(0,1000),
                    scope='north america',
                    labels={'Count':'Number of jobs'})
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# Number of jobs and Avg Salary by Province
df_sub = df[df['Province']!='Undef'].groupby('Province_FN').agg({'Avg_Salary': 'mean', 'Province': 'count'}).reset_index()
df_sub.columns = ['Province', 'Avg_Salary', 'Count']

fig = px.scatter(df_sub, x='Count', y='Avg_Salary', hover_name = 'Province',
                size='Count', color = 'Province', log_x=True, size_max=60,
                title = 'Number of jobs vs. Avg Salary by Province')
fig.show()

In [ ]:
# Number of jobs by Province and industry
df_sub = df[(df['Province']!='Undef')&
            (df['Industry Type'].isin(['Technology', 'Healthcare', 'Finance', 'Service', 'Consulting']))]
         
df_sub = df_sub.groupby(['Province_FN', 'Industry Type']).agg(
    Avg_Salary=('Avg_Salary', 'mean'),
    Count=('Province', 'count')).reset_index()

# Average Salary by Province and Industry Type
fig = px.bar(df_sub, 
             x='Province_FN', 
             y='Avg_Salary', 
             color='Industry Type', 
             barmode='group', 
             title='Average Salary by Province and Industry Type')

fig.show()

In [ ]:
# Number of jobs by Province and Seniority
df_sub = df[(df['Province']!='Undef')&
            (df['Seniority']!= 'ANY')]
         
df_sub = df_sub.groupby(['Province_FN', 'Seniority']).agg(
    Count=('Province', 'count')).reset_index()

# Create the Sunburst chart
fig = px.sunburst(df_sub, 
                  path=['Province_FN', 'Seniority'],  # Define the hierarchy path
                  values='Count', 
                  title='Job Distribution by Province and Seniority')

fig.show()

# 2.4. Avg Salary based on Skill Sets

In [ ]:
df['Skill_Set']=df['Skill'].str.split(',')
df_exploded = df.explode('Skill_Set')
df_exploded['Skill_Set']=df_exploded['Skill_Set'].str.upper().str.strip()
df_exploded = df_exploded.drop(columns=['Skill','Province_FN'])

In [ ]:
Count_Skill=df_exploded[df_exploded['Skill_Set']!='UNDEF']['Skill_Set'].value_counts().reset_index().sort_values(by='count', ascending=False)
Count_Skill

In [ ]:
fig = px.bar(Count_Skill.head(20).sort_values(by='count', ascending =True),
             x = 'count', y = 'Skill_Set', orientation='h', text_auto='.2s')
fig.update_layout(
    title = 'Distribution of Data Analyst Skill',
    yaxis_title= 'Skill',
    xaxis_title='Count',
    boxmode='group')
fig.update_traces(textfont_size=12, textangle=0, textposition='outside',cliponaxis=False)

fig.show()

In [ ]:
# Avg Salary by Skill set
df_exploded.head(5)
df_sub= df_exploded.groupby('Skill_Set').agg(Avg_Salary=('Avg_Salary','mean'), Count=('Skill_Set', 'count')).reset_index()
df_sub=df_sub.sort_values(by='Avg_Salary', ascending=False)
fig = px.scatter(df_sub, x='Count', y='Avg_Salary', hover_name='Skill_Set',
                size='Count', color='Skill_Set', log_x=True, size_max=60,
                title = 'Count of jobs vs Avg Salary by Skill')

fig.show()

# 2.5. Avg Salary by Industry Type

In [ ]:
Count_Industry = df[df['Industry Type'] != 'Others']['Industry Type'].value_counts().reset_index()
fig = px.pie(Count_Industry, values='count', names='Industry Type',
            title = 'Percentage of number of jobs by Industry Type')
fig.show()

In [ ]:
df_Industry = df.groupby('Industry Type').agg(Avg_Salary = ('Avg_Salary', 'mean')).reset_index()\
            .sort_values(by='Avg_Salary', ascending=False)
fig_Industry = go.Scatter(x = df_Industry['Industry Type'], y = df_Industry['Avg_Salary'], mode='markers')

Count_Seniority = df['Seniority'].value_counts().reset_index()
fig_coun_seniority= go.Bar(x=Count_Seniority['Seniority'], y=Count_Seniority['count'])


# Which industry pay highest salary for senior and junior level
df_senior=df[df['Seniority']=='Senior'].groupby(['Industry Type'])\
            .agg(Avg_Salary=('Avg_Salary', 'mean')).reset_index()\
            .sort_values(by='Avg_Salary', ascending=True)
df_junior=df[df['Seniority']=='Junior'].groupby(['Industry Type'])\
            .agg(Avg_Salary=('Avg_Salary', 'mean')).reset_index()\
            .sort_values(by='Avg_Salary', ascending=True)

fig_senior = go.Bar(y=df_senior['Industry Type'], x=df_senior['Avg_Salary'], name='Senior', orientation='h')
fig_junior = go.Bar(y=df_junior['Industry Type'], x=df_senior['Avg_Salary'], name='Junior',  orientation='h')

fig = make_subplots(rows=2, cols=2, subplot_titles=('Avg Salary by Industry Type','Count of Seniority Type', 'Senior Positions', 'Junior Positions'),
                    horizontal_spacing=0.15, vertical_spacing=0.25)
fig.add_trace(fig_Industry, row=1, col=1)
fig.add_trace(fig_coun_seniority, row=1, col=2)
fig.add_trace(fig_senior, row=2, col=1)
fig.add_trace(fig_junior, row=2, col=2)
fig.update_layout(height=800, width=1200, title_text='Industries pay highest salary for Senior and Junior Positions',
                 xaxis=dict(tickangle=-45))

fig.show()

# **3. MACHINE LEARNING MODEL - PREDICTION AVG SALARY**

In [ ]:
X = df_exploded.drop(columns=['Avg_Salary','Min_Salary', 'Max_Salary', 'Work Type', 'Seniority'])
y = df_exploded['Avg_Salary']

In [ ]:
import category_encoders as ce
categorical_features = ['Job Title', 'Job Info', 'Position', 'Employer', 'City', 'Province', 'Industry Type','Skill_Set']

# Example of Target Encoding
target_encoder = ce.TargetEncoder(cols=categorical_features)
X_encoded = target_encoder.fit_transform(X,y)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# 3.1 Random Forest Regressor Model

In [ ]:
# Define the model
model = RandomForestRegressor(n_estimators=30, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Model evaluation
y_pred = model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Absolute Error:{mae}')
print(f'Mean Squared Error:{mse}')
print(f'R-Squared:{r2}')

# 3.2. Important Feature

In [ ]:
important_features = model.feature_importances_
important_feature_df = pd.DataFrame({
                                    'Feature': X_train.columns,
                                    'Importance':important_features 
                                    }).sort_values(by='Importance', ascending=True)
fig = px.bar(important_feature_df, y = 'Feature', x='Importance', orientation='h',
            title = 'Feature Importance')
fig.show()

In [ ]:
# Focus on most importance feature
X =df_exploded[['Employer', 'Job Info', 'Skill_Set']]
y = df_exploded['Avg_Salary']

categorical_features = ['Employer', 'Job Info', 'Skill_Set']

# Example of Target Encoding
target_encoder = ce.TargetEncoder(cols=categorical_features)
X_encoded = target_encoder.fit_transform(X,y)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Define the model
model = RandomForestRegressor(n_estimators=30, random_state=42)
model.fit(X_train, y_train)

# Model evaluation
y_pred = model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Absolute Error:{mae}')
print(f'Mean Squared Error:{mse}')
print(f'R-Squared:{r2}')


# 3.3 Experiment with Hyperparameter Tuning

In [ ]:

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best Parameters: ", grid_search.best_params_)
print("Best Cross-Validation Score: ", grid_search.best_score_)

# Use the best model to predict
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)



In [ ]:
# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred_best)
mse = mean_squared_error(y_test, y_pred_best)
r2 = r2_score(y_test, y_pred_best)
print(f'Mean Absolute Error:{mae}')
print(f'Mean Squared Error:{mse}')
print(f'R-Squared:{r2}')
